
<br>
============<br>
Thresholding<br>
============<br>
Thresholding is used to create a binary image from a grayscale image [1]_.<br>
It is the simplest way to segment objects from a background.<br>
Thresholding algorithms implemented in scikit-image can be separated in two<br>
categories:<br>
- Histogram-based. The histogram of the pixels' intensity is used and<br>
  certain assumptions are made on the properties of this histogram (e.g. bimodal).<br>
- Local. To process a pixel, only the neighboring pixels are used.<br>
  These algorithms often require more computation time.<br>
If you are not familiar with the details of the different algorithms and the<br>
underlying assumptions, it is often difficult to know which algorithm will give<br>
the best results. Therefore, Scikit-image includes a function to evaluate<br>
thresholding algorithms provided by the library. At a glance, you can select<br>
the best algorithm for you data without a deep understanding of their<br>
mechanisms.<br>
.. [1] https://en.wikipedia.org/wiki/Thresholding_%28image_processing%29<br>
.. seealso::<br>
   Presentation on<br>
   :ref:`sphx_glr_auto_examples_applications_plot_rank_filters.py`.<br>


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from skimage import data
from skimage.filters import try_all_threshold

In [ ]:
img = data.page()

In [ ]:
fig, ax = try_all_threshold(img, figsize=(10, 8), verbose=False)
plt.show()

####################################################################<br>
How to apply a threshold?<br>
=========================<br>
<br>
Now, we illustrate how to apply one of these thresholding algorithms.<br>
This example uses the mean value of pixel intensities. It is a simple<br>
and naive threshold value, which is sometimes used as a guess value.<br>


In [ ]:
from skimage.filters import threshold_mean

In [ ]:
image = data.camera()
thresh = threshold_mean(image)
binary = image > thresh

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8, 3))
ax = axes.ravel()

In [ ]:
ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Original image')

In [ ]:
ax[1].imshow(binary, cmap=plt.cm.gray)
ax[1].set_title('Result')

In [ ]:
for a in ax:
    a.axis('off')

In [ ]:
plt.show()

####################################################################<br>
Bimodal histogram<br>
=================<br>
<br>
For pictures with a bimodal histogram, more specific algorithms can be used.<br>
For instance, the minimum algorithm takes a histogram of the image and smooths it<br>
repeatedly until there are only two peaks in the histogram.

In [ ]:
from skimage.filters import threshold_minimum

In [ ]:
image = data.camera()

In [ ]:
thresh_min = threshold_minimum(image)
binary_min = image > thresh_min

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))

In [ ]:
ax[0, 0].imshow(image, cmap=plt.cm.gray)
ax[0, 0].set_title('Original')

In [ ]:
ax[0, 1].hist(image.ravel(), bins=256)
ax[0, 1].set_title('Histogram')

In [ ]:
ax[1, 0].imshow(binary_min, cmap=plt.cm.gray)
ax[1, 0].set_title('Thresholded (min)')

In [ ]:
ax[1, 1].hist(image.ravel(), bins=256)
ax[1, 1].axvline(thresh_min, color='r')

In [ ]:
for a in ax[:, 0]:
    a.axis('off')
plt.show()

####################################################################<br>
Otsu's method [2]_ calculates an "optimal" threshold (marked by a red line in the<br>
histogram below) by maximizing the variance between two classes of pixels,<br>
which are separated by the threshold. Equivalently, this threshold minimizes<br>
the intra-class variance.<br>
<br>
.. [2] https://en.wikipedia.org/wiki/Otsu's_method<br>


In [ ]:
from skimage.filters import threshold_otsu

In [ ]:
image = data.camera()
thresh = threshold_otsu(image)
binary = image > thresh

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(8, 2.5))
ax = axes.ravel()
ax[0] = plt.subplot(1, 3, 1)
ax[1] = plt.subplot(1, 3, 2)
ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

In [ ]:
ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Original')
ax[0].axis('off')

In [ ]:
ax[1].hist(image.ravel(), bins=256)
ax[1].set_title('Histogram')
ax[1].axvline(thresh, color='r')

In [ ]:
ax[2].imshow(binary, cmap=plt.cm.gray)
ax[2].set_title('Thresholded')
ax[2].axis('off')

In [ ]:
plt.show()

####################################################################<br>
Local thresholding<br>
==================<br>
<br>
If the image background is relatively uniform, then you can use a global<br>
threshold value as presented above. However, if there is large variation in the<br>
background intensity, adaptive thresholding (a.k.a. local or dynamic<br>
thresholding) may produce better results. Note that local is much slower than<br>
global thresholding.<br>
<br>
Here, we binarize an image using the `threshold_local` function, which<br>
calculates thresholds in regions with a characteristic size `block_size` surrounding<br>
each pixel (i.e. local neighborhoods). Each threshold value is the weighted mean<br>
of the local neighborhood minus an offset value.<br>


In [ ]:
from skimage.filters import threshold_otsu, threshold_local

In [ ]:
image = data.page()

In [ ]:
global_thresh = threshold_otsu(image)
binary_global = image > global_thresh

In [ ]:
block_size = 35
local_thresh = threshold_local(image, block_size, offset=10)
binary_local = image > local_thresh

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(7, 8))
ax = axes.ravel()
plt.gray()

In [ ]:
ax[0].imshow(image)
ax[0].set_title('Original')

In [ ]:
ax[1].imshow(binary_global)
ax[1].set_title('Global thresholding')

In [ ]:
ax[2].imshow(binary_local)
ax[2].set_title('Local thresholding')

In [ ]:
for a in ax:
    a.axis('off')

In [ ]:
plt.show()

####################################################################<br>
Now, we show how Otsu's threshold [2]_ method can be applied locally. For<br>
each pixel, an "optimal" threshold is determined by maximizing the variance<br>
between two classes of pixels of the local neighborhood defined by a<br>
structuring element.<br>
<br>
The example compares the local threshold with the global threshold.<br>


In [ ]:
from skimage.morphology import disk
from skimage.filters import threshold_otsu, rank
from skimage.util import img_as_ubyte

In [ ]:
img = img_as_ubyte(data.page())

In [ ]:
radius = 15
selem = disk(radius)

In [ ]:
local_otsu = rank.otsu(img, selem)
threshold_global_otsu = threshold_otsu(img)
global_otsu = img >= threshold_global_otsu

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 5), sharex=True, sharey=True)
ax = axes.ravel()
plt.tight_layout()

In [ ]:
fig.colorbar(ax[0].imshow(img, cmap=plt.cm.gray),
             ax=ax[0], orientation='horizontal')
ax[0].set_title('Original')
ax[0].axis('off')

In [ ]:
fig.colorbar(ax[1].imshow(local_otsu, cmap=plt.cm.gray),
             ax=ax[1], orientation='horizontal')
ax[1].set_title('Local Otsu (radius=%d)' % radius)
ax[1].axis('off')

In [ ]:
ax[2].imshow(img >= local_otsu, cmap=plt.cm.gray)
ax[2].set_title('Original >= Local Otsu' % threshold_global_otsu)
ax[2].axis('off')

In [ ]:
ax[3].imshow(global_otsu, cmap=plt.cm.gray)
ax[3].set_title('Global Otsu (threshold = %d)' % threshold_global_otsu)
ax[3].axis('off')

In [ ]:
plt.show()